In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
sys.path.insert(0, '../../src/')
import data_loader
pd.set_option('display.max_columns', None)

In [2]:
season_2018 = data_loader.load_data(seasons=["2018-19"])

In [3]:
seasons_ldg = data_loader.load_game_data(seasons=["2018-19"])

Loaded PBP-data


In [4]:
seasons_ldg.loc[21800794]

play_count                                449
season_name                           2018-19
visitor_team_id                  1610612745.0
visitor_team_city                     Houston
visitor_team_nickname                 Rockets
visitor_final_score                       118
home_final_score                          110
home_win                                False
home_team_id                     1610612756.0
home_team_city                        Phoenix
home_team_nickname                       Suns
periods                                     4
minutes_played                           48.0
visitor_players_deployed                    8
home_players_deployed                       8
visitor_fg_made                            43
visitor_fg_missed                          44
visitor_3PT_made                           13
visitor_3PT_missed                         28
home_fg_made                               45
home_fg_missed                             39
home_3PT_made                     

In [5]:
season_2018

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name
0,0,0,PERIOD_BEGIN,2,21800794,NaN,12:00,1,0.0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:11 PM,2018-19
1,1,0,JUMP_BALL,4,21800794,Jump Ball Ayton vs. Faried: Tip to Okobo,12:00,1,HOME_PLAYER,VISITOR_PLAYER,HOME_PLAYER,1629028,Deandre Ayton,PHX,Phoenix,1610612756.0,Suns,202702,Kenneth Faried,HOU,Houston,1610612745.0,Rockets,1629059,Elie Okobo,PHX,Phoenix,1610612756.0,Suns,NaN,NaN,NaN,9:11 PM,2018-19
2,2,1,TURNOVER,7,21800794,Jackson Bad Pass Turnover (P1.T1),11:39,1,HOME_PLAYER,VISITOR_PLAYER,0,1628367,Josh Jackson,PHX,Phoenix,1610612756.0,Suns,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Harden STEAL (1 STL),9:11 PM,2018-19
3,3,Driving Layup,FIELD_GOAL_MADE,9,21800794,NaN,11:31,1,VISITOR_PLAYER,0,0,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2 - 0,-2,Harden 3' Driving Layup (2 PTS),9:12 PM,2018-19
4,4,2,FOUL,10,21800794,Bridges S.FOUL (P1.T1) (E.Dalen),11:31,1,HOME_PLAYER,VISITOR_PLAYER,TIMEOUT,1628969,Mikal Bridges,PHX,Phoenix,1610612756.0,Suns,201935,James Harden,HOU,Houston,1610612745.0,Rockets,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9:12 PM,2018-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582463,431,1,TIMEOUT,634,21801037,NaN,0:04,4,VISITOR_TEAM,0,0,1610612739,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cavaliers Timeout: Regular (Reg.6 Short 0),10:49 PM,2018-19
582464,432,0,SUBSTITUTION,635,21801037,NaN,0:04,4,VISITOR_PLAYER,VISITOR_PLAYER,0,1627790,Ante Zizic,CLE,Cleveland,1610612739.0,Cavaliers,203903,Jordan Clarkson,CLE,Cleveland,1610612739.0,Cavaliers,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Clarkson FOR Zizic,10:50 PM,2018-19
582465,433,Fadeaway Jumper,FIELD_GOAL_MISSED,638,21801037,NaN,0:02,4,VISITOR_PLAYER,0,0,203917,Nik Stauskas,CLE,Cleveland,1610612739.0,Cavaliers,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISS Stauskas 24' 3PT Fadeaway Jumper,10:51 PM,2018-19
582466,434,live,REBOUND,639,21801037,Kleber REBOUND (Off:5 Def:7),0:02,4,HOME_PLAYER,0,0,1628467,Maxi Kleber,DAL,Dallas,1610612742.0,Mavericks,0,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:51 PM,2018-19


In [10]:
# Number of Phoenix Suns players that were substituted in the first period
num_of_substitutions_1 = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER1_TEAM_ID"] == "1610612756.0")].groupby("GAME_ID")["PLAYER1_NAME"].nunique()

In [162]:
# Phoenix Suns players that were substituted in the first period
season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["PERIOD"] == 1) & (season_2018["PLAYER1_TEAM_ID"] == "1610612756.0")].groupby("GAME_ID")["PLAYER1_NAME"].unique()


GAME_ID
21800013    [Ryan Anderson, Devin Booker, Deandre Ayton, I...
21800033    [Ryan Anderson, Deandre Ayton, Isaiah Canaan, ...
21800047    [Deandre Ayton, Ryan Anderson, Devin Booker, I...
21800060    [Ryan Anderson, Devin Booker, Deandre Ayton, I...
21800080                       [Ryan Anderson, Isaiah Canaan]
                                  ...                        
21801160    [Dragan Bender, Richaun Holmes, Mikal Bridges,...
21801174    [Dragan Bender, Mikal Bridges, Devin Booker, R...
21801189            [Ray Spalding, Mikal Bridges, Elie Okobo]
21801200    [Ray Spalding, Mikal Bridges, Elie Okobo, Josh...
21801216    [Ray Spalding, Mikal Bridges, Elie Okobo, Josh...
Name: PLAYER1_NAME, Length: 82, dtype: object

In [161]:
# Phoenix Suns players that were substituted in the whole match
season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["PLAYER1_TEAM_ID"] == "1610612756.0")].groupby("GAME_ID")["PLAYER1_NAME"].unique()


GAME_ID
21800013    [Ryan Anderson, Devin Booker, Deandre Ayton, I...
21800033    [Ryan Anderson, Deandre Ayton, Isaiah Canaan, ...
21800047    [Deandre Ayton, Ryan Anderson, Devin Booker, I...
21800060    [Ryan Anderson, Devin Booker, Deandre Ayton, I...
21800080    [Ryan Anderson, Isaiah Canaan, Trevor Ariza, E...
                                  ...                        
21801160    [Dragan Bender, Richaun Holmes, Mikal Bridges,...
21801174    [Dragan Bender, Mikal Bridges, Devin Booker, R...
21801189    [Ray Spalding, Mikal Bridges, Elie Okobo, Troy...
21801200    [Ray Spalding, Mikal Bridges, Elie Okobo, Josh...
21801216    [Ray Spalding, Mikal Bridges, Elie Okobo, Josh...
Name: PLAYER1_NAME, Length: 82, dtype: object

## TESTING - STARTING LINEUPS

In [ ]:
def starters():
    GAME_ID_STR = "GAME_ID"

    # TODO -> load season_data + load game_data



    pbp_grouped = season_2018.groupby(GAME_ID_STR)
    game_data = pbp_grouped.size().to_frame(name="play_count")

    # assign ID to home and away team
    game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
    
    for i, row in game_data.iterrows():
        game_id = i
        home_team_id = row["HOME TEAM ID"]
        visitor_team_id = row["VISITOR TEAM ID"]
        
        home_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == game_id) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_NAME"].unique()
        print(game_id)

        if len(home_players) >= 5:
            game_data.at[game_id, "HOME_PLAYER_1":"HOME_PLAYER_5"] = home_players[:5]




In [11]:
GAME_ID_STR = "GAME_ID"
pbp_grouped = season_2018.groupby(GAME_ID_STR)

In [12]:
game_data = pbp_grouped.size().to_frame(name="play_count")

In [18]:
game_data["HOME TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'HOME_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])
game_data["VISITOR TEAM ID"] = pbp_grouped.apply(lambda x: x[x["PERSON1TYPE"] == 'VISITOR_PLAYER']["PLAYER1_TEAM_ID"].iloc[0])

In [141]:
game_data[game_data["HOME_PLAYER_2"] == 0]

,play_count,HOME TEAM ID,VISITOR TEAM ID,HOME_PLAYER_1,HOME_PLAYER_2,HOME_PLAYER_3,HOME_PLAYER_4,HOME_PLAYER_5
GAME_ID,,,,,,,,
21800902,471.0,1610612741.0,1610612749.0,Robin Lopez,0,0,0,0


In [127]:
game_data[["HOME_PLAYER_1", "HOME_PLAYER_2", "HOME_PLAYER_3", "HOME_PLAYER_4", "HOME_PLAYER_5"]] = np.nan

### TESTING - STARTING LINEUPS

In [130]:
for i, row in game_data.iterrows():
    game_id = i
    home_team_id = row["HOME TEAM ID"]
    visitor_team_id = row["VISITOR TEAM ID"]

    
    home_players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == game_id) & (season_2018["PERIOD"] == 1) &(season_2018["PLAYER1_TEAM_ID"] == home_team_id)]["PLAYER1_NAME"].unique()
    print(game_id)

    if len(home_players) < 5:
        home_players.resize(5)
        
    game_data.at[game_id, "HOME_PLAYER_1":"HOME_PLAYER_5"] = home_players[:5]


21800001
21800002
21800003
21800004
21800005
21800006
21800007
21800008
21800009
21800010
21800011
21800012
21800013
21800014
21800015
21800016
21800017
21800018
21800019
21800020
21800021
21800022
21800023
21800024
21800025
21800026
21800027
21800028
21800029
21800030
21800031
21800032
21800033
21800034
21800035
21800036
21800037
21800038
21800039
21800040
21800041
21800042
21800043
21800044
21800045
21800046
21800047
21800048
21800049
21800050
21800051
21800052
21800053
21800054
21800055
21800056
21800057
21800058
21800059
21800060
21800061
21800062
21800063
21800064
21800065
21800066
21800067
21800068
21800069
21800070
21800071
21800072
21800073
21800074
21800075
21800076
21800077
21800078
21800079
21800080
21800081
21800082
21800083
21800084
21800085
21800086
21800087
21800088
21800089
21800090
21800091
21800092
21800093
21800094
21800095
21800096
21800097
21800098
21800099
21800100
21800101
21800102
21800103
21800104
21800105
21800106
21800107
21800108
21800109
21800110
21800111
2

In [142]:
players = season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == 21800902) & (season_2018["PERIOD"] == 1) & (season_2018["PERSON1TYPE"] == 'HOME_PLAYER')]["PLAYER1_NAME"].unique()
players

array(['Robin Lopez'], dtype=object)

In [143]:
season_2018[(season_2018["EVENTMSGTYPE"] == "SUBSTITUTION") & (season_2018["GAME_ID"] == 21800902) & (season_2018["PERIOD"] == 1) & (season_2018["PERSON1TYPE"] == 'HOME_PLAYER')]

,Unnamed: 0,EVENTMSGACTIONTYPE,EVENTMSGTYPE,EVENTNUM,GAME_ID,HOMEDESCRIPTION,PCTIMESTRING,PERIOD,PERSON1TYPE,PERSON2TYPE,PERSON3TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ABBREVIATION,PLAYER1_TEAM_CITY,PLAYER1_TEAM_ID,PLAYER1_TEAM_NICKNAME,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ABBREVIATION,PLAYER2_TEAM_CITY,PLAYER2_TEAM_ID,PLAYER2_TEAM_NICKNAME,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ABBREVIATION,PLAYER3_TEAM_CITY,PLAYER3_TEAM_ID,PLAYER3_TEAM_NICKNAME,SCORE,SCOREMARGIN,VISITORDESCRIPTION,WCTIMESTRING,season_name
106701,82,0,SUBSTITUTION,116,21800902,SUB: Selden FOR Lopez,3:06,1,HOME_PLAYER,HOME_PLAYER,0,201577,Robin Lopez,CHI,Chicago,1610612741.0,Bulls,1627782,Wayne Selden,CHI,Chicago,1610612741.0,Bulls,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8:29 PM,2018-19
